# MiBiPreT example: Ordination with Amersfoort data

Diagnostic plots for data analysis on microbial biodegredation at the contaminant Amersfoort site. Author: Alraune Zech

Data based on the PhD thesis of *Johan van Leeuwen*, 2021 'Biodegredation of mono- and polyaromatic hydrocarbons in a contaminated aquifer originating from a former Pintsch gas factory site' which is equivalent to the manuscript of van Leeuwen et al., 2022 'Anaerobic degradation of benzene and other aromatic hydrocarbons in a tar-derived plume: Nitrate versus iron reducing conditions', J. of Cont. Hydrol. The data was provided by Johan van Leeuwen.
  
## Background: Amersfoort contaminant site

Close to the train station in Amersfoort, the Netherlands, the subsurface is contaminated with organic hydrocarbons forming a NAPL. Contamination originates from decades of operating manufactured gas plant, which dumped tar by-products in waste lagoons. The tar is a DNAPL and has spread into the underlying shallow unconfined aquifer. Sample wells were installed to measure various characteristics of the subsurface. The raw data contains measurements on
* environmental conditions, such as pH, redox potential, concentrations of oxygen, nitrate, etc
* contaminant concentration such as BTEX, indene, indane, naphtalene and multiple other (typically cyclic) petroleum hydrocarbons
* metabolite concentration, i.e. byproducts of degredation processes of contaminant
* isotope measurments for specific contaminants and samples
* counts of genes (RNA/DNA) of mibrobiota that is know to perform biodegredation as well as functional enzymes know to be responsible for biodegredation

**Required packages**

In [ ]:
import mibiscreen as mbs

## Data loading

In [ ]:
file_path = './amersfoort.xlsx'

Load and standardize data of environmental quantities/chemicals

In [ ]:
environment_raw,units = mbs.load_excel(file_path,
                                    sheet_name = 'environment',
                                    verbose = False)

environment,units = mbs.standardize(environment_raw,
                                reduce = True,
                                verbose=False)

Load and standardize data of contaminants:

In [ ]:
contaminants_raw,units = mbs.load_excel(file_path,
                                    sheet_name = 'contaminants',
                                    verbose = False)

contaminants,units = mbs.standardize(contaminants_raw,
                                  reduce = False,
                                  verbose = False)

mbs.total_contaminant_concentration(contaminants,
                                include = True,
                                verbose = False)

Load and standardize data of DNA related data:

In [ ]:
###------------------------------------------------------------------------###
### Load and standardize data of contaminants
dna_raw,units = mbs.load_excel(file_path,
                           sheet_name = 'sequencing',
                           verbose = True)

dna,units = mbs.standardize(dna_raw,
                        reduce = False,
                        verbose = True)

Merge both data frames into one for further analysis:

In [ ]:
data = mbs.merge_data([environment,contaminants],clean = True)

## Unconstrained Analysis (PCA)
### Data pre-processing

Specification of variables to chose for ordination. We make sure that variables selected for analysis are in standard names.

In [ ]:
variables_1 = mbs.standard_names(['total_contaminants'])
variables_2 = mbs.standard_names(['nitrate','pH','nitrite','sulfate','Redox','EC','DOC',"Mn2","Fe2"])

Extracting data of interest from data frame. All other quantities are removed, since only columns of interest should be passed to ordination analysis routines. 

In [ ]:
data_ordination = mbs.extract_data(data,
                               name_list = variables_1 + variables_2,
                               keep_setting_data = True)

Cleaning data, i.e. remove all sites where measurements are missing for any of the quantities

In [ ]:
data_filtered = mbs.filter_values(data_ordination, 
              replace_NaN = 'remove', 
              inplace = True,
              verbose = True)
data_filtered

### Data transformation

Concentrations of contaminants (`Sum GC`) is log-transformed (given that concentration vary over orders of magnitude) and then standardized.
Other data is only standardized.

In [ ]:
data_transformed = mbs.transform_values(data_ordination,
                 name_list = variables_1,
                 how = 'log_scale',
                 inplace = True,
                 )

data_transformed = mbs.transform_values(data_ordination,
                  name_list = variables_1,
                  how = 'standardize',
                  inplace = True,
                  )

data_transformed = mbs.transform_values(data_ordination,
                  name_list = variables_2,
                  how = 'standardize',
                  inplace = True,
                )

### Ordination analysis by PCA

In [ ]:
ordination_output = mbs.pca(data_ordination,
                        independent_variables = variables_1+variables_2,
                        verbose = True)

### Ordination plot

In [ ]:
fig, ax = mbs.ordination_plot(ordination_output=ordination_output,
                plot_scores = True, 
                plot_loadings = True,
                rescale_loadings_scores = True,
                figsize = [4.5,3],
                title = "Unconstrained Ordination PCA",                
                axis_ranges = [-0.6,0.8,-0.6,0.7],
                label_fontsize = 10,
                loading_fontsize = 8,
                #score_fontsize = 6,
                #save_fig = 'ordination_amersfoort_pca.png',
                )

### Interpretation

The figure shows the results of the PCA (principle component analysis) as multi-variate statistics tool for the geochemical parameters. The axes denote the percentage of variance explained by each principal component. Data was standardized, with `Sum GC` (being the total sum of contaminants) being logarithmically transformed as via $log(x+1)$.

This biplot shows two different elements of information: the variable loadings show as arrows and the site scores shown as dots. The two plot axis represent the two main axis identified by the ordination methods. They are scales to the minimal value of $-1$ and the maximum value of $1$. 

The direction of the arrow indicates to which ordination axis it correlates. The length of the vector is equivalent to the extent of that correlation. Positive scores or loadings indicate positive correlation with the axis, where negative values indicate negative 
correlation. For example, a variable with negative loadings for the first two ordination axes is anticorrelated with the two largest trends in the data. Arrows pointing in the same direction indicate that the variable are correlated. Arrows at an right angle to one another are uncorrelated. Arrows that point in opposite directions are anti-correlated. A vector very close to the origin shows little to no correlation with the axes. 

Proximity of the site scores in the plot indicate the similarity between the sample sites. 

## Constrained Analysis (CCA & RDA)
### Data preprocessing

Specify groups of quantities to perform analysis on:
* group of geochemical parameters
* group of contaminants
* sequencing data

In [ ]:
geochemicals_group = ['nitrate','sulfate','redoxpot','iron2','manganese2']
#contaminant_group = ['benzene','toluene','ethylbenzene','pm_xylene','indene','naphthalene']
contaminant_group = ['benzene','toluene','indene','naphthalene']
variables_dna = ['Total bacteria 16SRrna', 'Benzene carboxylase', 'NirS', 'NarG', 'BssA SRB',
                 'BssA nitraat', 'Peptococcus']

Extract data of quantities of interest:
*geochemical
*contaminants

In [ ]:
geochem_selected = environment[['sample_nr']+geochemicals_group]
cont_selected = contaminants[['sample_nr']+contaminant_group]

contaminant_group_analysis = list(cont_selected.columns)
contaminant_group_analysis.remove('sample_nr')

Concatenate all relevant data:

In [ ]:
data_ordination = mbs.merge_data([geochem_selected,cont_selected,dna],clean = True)


### Data transformation for CCA

In [ ]:
data_cca = data_ordination.copy()

mbs.filter_values(data_cca,
              replace_NaN = 'remove',#'average',  # #'zero'
              inplace = True,
              #verbose = True
                 )

mbs.transform_values(data_cca,
                 name_list = variables_dna,
                 how = 'log_scale',
                 inplace = True,
                 )

### Ordination analysis by CCA

In [ ]:
cca_output = mbs.cca(data_cca,
                  independent_variables = contaminant_group_analysis,
                  dependent_variables = variables_dna,
                  verbose = True)

### plot results CCA

In [ ]:
fig, ax = mbs.ordination_plot(ordination_output=cca_output,
                plot_scores = False,
                plot_loadings = True,
                rescale_loadings_scores = False,
                label_fontsize = 10,
                loading_fontsize = 8,
                score_fontsize = 6,
                title ="Constrained Ordination CCA",
                save_fig = 'cca_dna.png',
                )

### Data transformation for RDA

In [ ]:
data_rda = data_ordination.copy()

mbs.filter_values(data_rda,
              replace_NaN = 'zero',
              # replace_NaN = 'average',
              # replace_NaN = 'remove',
              inplace = True,
              verbose = True)

### if (part of) data is log-transformed (before standardization):
mbs.transform_values(data_rda,
                  name_list = variables_dna,
                  how = 'log_scale',
                  inplace = True,
                  )

mbs.transform_values(data_rda,
                  how = 'standardize',
                  inplace = True,
                  )


### Ordination analysis by RDA

In [ ]:
rda_output = mbs.rda(data_rda,
                  independent_variables = contaminant_group_analysis,
                  # independent_variables = contaminant_group_analysis + geochemicals_group,
                  dependent_variables = variables_dna,
                  verbose = True)


### plot results RDA

In [ ]:
fig, ax = mbs.ordination_plot(ordination_output=rda_output,
                plot_scores = False,
                plot_loadings = True,
                rescale_loadings_scores = False,
                title = "Constrained Ordination RDA",
                figsize = [4.5,3],
                label_fontsize = 10,
                loading_fontsize = 8,
                #score_fontsize = 6,
                # axis_ranges = [-0.6,0.8,-0.8,1.0],
                save_fig = 'ordination_amersfoort_rda.png',
                )